In [45]:
import pandas as pd

import os
from sqlalchemy import create_engine , text
from sqlalchemy.orm import sessionmaker
import mysql.connector

In [46]:
# Configure the connection to your MySQL database
engine = create_engine('mysql+mysqlconnector://root:@localhost/airbnb')

In [40]:
# import csv data from Inside Airbnb " http://insideairbnb.com/get-the-data/ "

df = pd.read_csv("listings.csv")


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23142 entries, 0 to 23141
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            23142 non-null  int64  
 1   listing_url                                   23142 non-null  object 
 2   scrape_id                                     23142 non-null  int64  
 3   last_scraped                                  23142 non-null  object 
 4   source                                        23142 non-null  object 
 5   name                                          23142 non-null  object 
 6   description                                   22744 non-null  object 
 7   neighborhood_overview                         12481 non-null  object 
 8   picture_url                                   23142 non-null  object 
 9   host_id                                       23142 non-null 

In [42]:
df = df.fillna('')

In [43]:
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6400,https://www.airbnb.com/rooms/6400,20230621232643,2023-06-22,city scrape,Rental unit in Milan · ★4.89 · 3 bedrooms · 1 ...,"Enjoy your stay at The Studio, a light-filled ...",The neighborhood is quiet and very well connec...,https://a0.muscache.com/pictures/474737/5955ba...,13822,...,5.0,4.56,4.67,,f,1,0,1,0,0.07
1,23986,https://www.airbnb.com/rooms/23986,20230621232643,2023-06-22,city scrape,Rental unit in Milan · ★4.63 · 1 bedroom · 1 b...,I look forward to welcoming you in my flat; it...,,https://a0.muscache.com/pictures/623d63f8-56cf...,95941,...,4.72,4.68,4.44,,f,1,1,0,0,0.18
2,24107,https://www.airbnb.com/rooms/24107,20230621232643,2023-06-22,city scrape,Condo in Milan · ★4.50 · 1 bedroom · 6 beds · ...,Un grazioso bilocale completamente rinnovato i...,,https://a0.muscache.com/pictures/cd4112f3-37c2...,46951,...,5.0,3.67,4.67,,f,1,1,0,0,0.02
3,40470,https://www.airbnb.com/rooms/40470,20230621232643,2023-06-22,city scrape,Rental unit in Milan · ★4.66 · 2 bedrooms · 4 ...,<b>The space</b><br />If you want to visit Mil...,,https://a0.muscache.com/pictures/891684/01c17b...,174203,...,4.87,4.39,4.5,,f,2,2,0,0,0.27
4,46536,https://www.airbnb.com/rooms/46536,20230621232643,2023-06-22,city scrape,Rental unit in Milan · ★4.59 · 2 bedrooms · 3 ...,Over the international fair in April we rent f...,"Flat It's located in north side of milan, jus...",https://a0.muscache.com/pictures/4eb8e0f5-e17b...,138683,...,4.93,4.38,4.66,,f,1,1,0,0,0.21


In [47]:
# Connection to MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='airbnb'
)

In [48]:
cursor = conn.cursor()


In [49]:
try:
    # Creating the table in the database based on the columns in the DataFrame
    create_table_query = f"CREATE TABLE IF NOT EXISTS milan ({', '.join([f'{col} TEXT' for col in df.columns])})"
    cursor.execute(create_table_query)

    # Insert data into the database
    for _, row in df.iterrows():
        insert_query = f"INSERT INTO milan ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"
        values = tuple(row)
        cursor.execute(insert_query, values)

    # Conferm
    conn.commit()

except Exception as e:
    # Undo changes if you make an error
    conn.rollback()
    print("Error occured:", str(e))

finally:
    # Closing the cursor and the connection
    cursor.close()
    conn.close()

In [50]:
## Check table

pd.read_sql('SELECT * FROM milan', engine)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6400,https://www.airbnb.com/rooms/6400,20230621232643,2023-06-22,city scrape,Rental unit in Milan · ★4.89 · 3 bedrooms · 1 ...,"Enjoy your stay at The Studio, a light-filled ...",The neighborhood is quiet and very well connec...,https://a0.muscache.com/pictures/474737/5955ba...,13822,...,5.0,4.56,4.67,,f,1,0,1,0,0.07
1,23986,https://www.airbnb.com/rooms/23986,20230621232643,2023-06-22,city scrape,Rental unit in Milan · ★4.63 · 1 bedroom · 1 b...,I look forward to welcoming you in my flat; it...,,https://a0.muscache.com/pictures/623d63f8-56cf...,95941,...,4.72,4.68,4.44,,f,1,1,0,0,0.18
2,24107,https://www.airbnb.com/rooms/24107,20230621232643,2023-06-22,city scrape,Condo in Milan · ★4.50 · 1 bedroom · 6 beds · ...,Un grazioso bilocale completamente rinnovato i...,,https://a0.muscache.com/pictures/cd4112f3-37c2...,46951,...,5.0,3.67,4.67,,f,1,1,0,0,0.02
3,40470,https://www.airbnb.com/rooms/40470,20230621232643,2023-06-22,city scrape,Rental unit in Milan · ★4.66 · 2 bedrooms · 4 ...,<b>The space</b><br />If you want to visit Mil...,,https://a0.muscache.com/pictures/891684/01c17b...,174203,...,4.87,4.39,4.5,,f,2,2,0,0,0.27
4,46536,https://www.airbnb.com/rooms/46536,20230621232643,2023-06-22,city scrape,Rental unit in Milan · ★4.59 · 2 bedrooms · 3 ...,Over the international fair in April we rent f...,"Flat It's located in north side of milan, jus...",https://a0.muscache.com/pictures/4eb8e0f5-e17b...,138683,...,4.93,4.38,4.66,,f,1,1,0,0,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23137,917286381468465380,https://www.airbnb.com/rooms/917286381468465380,20230621232643,2023-06-22,city scrape,Rental unit in Milano · ★New · 1 bedroom · 1 b...,L'appartamento è un bellissimo bilocale di rin...,,https://a0.muscache.com/pictures/miso/Hosting-...,497775861,...,,,,,t,3,3,0,0,
23138,915023142785359078,https://www.airbnb.com/rooms/915023142785359078,20230621232643,2023-06-22,city scrape,Rental unit in Milano · ★New · 1 bedroom · 1 b...,Appartamento di recente costruzione a 3 min ca...,,https://a0.muscache.com/pictures/miso/Hosting-...,405312215,...,,,,,t,1,1,0,0,
23139,917286635554478737,https://www.airbnb.com/rooms/917286635554478737,20230621232643,2023-06-22,city scrape,Rental unit in Milano · ★New · 3 bedrooms · 5 ...,"A disposizione un'appartamento enorme, luminos...",Il quartiere gode di un'atmosfera tranquilla e...,https://a0.muscache.com/pictures/707cbcf7-5c2d...,420375498,...,,,,,t,2,2,0,0,
23140,915025934511019523,https://www.airbnb.com/rooms/915025934511019523,20230621232643,2023-06-22,city scrape,Rental unit in Milano · ★New · 2 bedrooms · 6 ...,Relax in this centrally located quiet space.<b...,"From an architectural point of view, the area ...",https://a0.muscache.com/pictures/miso/Hosting-...,4015119,...,,,,,f,49,49,0,0,


In [ ]:
# top 10 most profitable on monthly basis

In [51]:
pd.read_sql('SELECT id, listing_url, name, host_name, availability_30 , price FROM milan', engine)

,id,listing_url,name,host_name,availability_30,price
0,6400,https://www.airbnb.com/rooms/6400,Rental unit in Milan · ★4.89 · 3 bedrooms · 1 ...,Francesca,23,$100.00
1,23986,https://www.airbnb.com/rooms/23986,Rental unit in Milan · ★4.63 · 1 bedroom · 1 b...,Jeremy,3,$150.00
2,24107,https://www.airbnb.com/rooms/24107,Condo in Milan · ★4.50 · 1 bedroom · 6 beds · ...,Valeria,30,$100.00
3,40470,https://www.airbnb.com/rooms/40470,Rental unit in Milan · ★4.66 · 2 bedrooms · 4 ...,Giacinto,22,$80.00
4,46536,https://www.airbnb.com/rooms/46536,Rental unit in Milan · ★4.59 · 2 bedrooms · 3 ...,Nico&Cinzia,0,$90.00
...,...,...,...,...,...,...
23137,917286381468465380,https://www.airbnb.com/rooms/917286381468465380,Rental unit in Milano · ★New · 1 bedroom · 1 b...,Simone,12,$61.00
23138,915023142785359078,https://www.airbnb.com/rooms/915023142785359078,Rental unit in Milano · ★New · 1 bedroom · 1 b...,Edoardo,15,$89.00
23139,917286635554478737,https://www.airbnb.com/rooms/917286635554478737,Rental unit in Milano · ★New · 3 bedrooms · 5 ...,Simone,30,$300.00
23140,915025934511019523,https://www.airbnb.com/rooms/915025934511019523,Rental unit in Milano · ★New · 2 bedrooms · 6 ...,Igor,20,$289.00


In [ ]:
# availability_30 means how many days the house still has no guests so you have to subtract it from 30 days in a month
# also $ sign in price column must be removed

In [92]:
conn = engine.connect()
query = text("UPDATE milan SET price = CAST(SUBSTRING(price, 2) AS SIGNED)")
conn.execute(query)
conn.close()

In [96]:
#price fixed
pd.read_sql('SELECT id, listing_url, name, host_name, availability_30 , price FROM milan', engine)

,id,listing_url,name,host_name,availability_30,price
0,6400,https://www.airbnb.com/rooms/6400,Rental unit in Milan · ★4.89 · 3 bedrooms · 1 ...,Francesca,23,100
1,23986,https://www.airbnb.com/rooms/23986,Rental unit in Milan · ★4.63 · 1 bedroom · 1 b...,Jeremy,3,150
2,24107,https://www.airbnb.com/rooms/24107,Condo in Milan · ★4.50 · 1 bedroom · 6 beds · ...,Valeria,30,100
3,40470,https://www.airbnb.com/rooms/40470,Rental unit in Milan · ★4.66 · 2 bedrooms · 4 ...,Giacinto,22,80
4,46536,https://www.airbnb.com/rooms/46536,Rental unit in Milan · ★4.59 · 2 bedrooms · 3 ...,Nico&Cinzia,0,90
...,...,...,...,...,...,...
23137,917286381468465380,https://www.airbnb.com/rooms/917286381468465380,Rental unit in Milano · ★New · 1 bedroom · 1 b...,Simone,12,61
23138,915023142785359078,https://www.airbnb.com/rooms/915023142785359078,Rental unit in Milano · ★New · 1 bedroom · 1 b...,Edoardo,15,89
23139,917286635554478737,https://www.airbnb.com/rooms/917286635554478737,Rental unit in Milano · ★New · 3 bedrooms · 5 ...,Simone,30,300
23140,915025934511019523,https://www.airbnb.com/rooms/915025934511019523,Rental unit in Milano · ★New · 2 bedrooms · 6 ...,Igor,20,289


In [11]:
# Just a check to see who already as no more free rooms
pd.read_sql('SELECT id, listing_url, name, host_name, availability_30 , price FROM milan WHERE availability_30 = 0', engine)

,id,listing_url,name,host_name,availability_30,price
0,46536,https://www.airbnb.com/rooms/46536,Rental unit in Milan · ★4.59 · 2 bedrooms · 3 ...,Nico&Cinzia,0,90
1,55055,https://www.airbnb.com/rooms/55055,Rental unit in Milano · 2 bedrooms · 1 bed · 1...,Cristina,0,145
2,93025,https://www.airbnb.com/rooms/93025,Rental unit in Milan · ★4.86 · 1 bedroom · 2 b...,Angela,0,90
3,94233,https://www.airbnb.com/rooms/94233,Rental unit in Milan · ★4.94 · 1 bedroom · 2 b...,Andre E Ale,0,95
4,74835,https://www.airbnb.com/rooms/74835,Rental unit in Milan · ★5.0 · 1 bedroom · 1 be...,Augusto,0,280
...,...,...,...,...,...,...
7965,918004230955889147,https://www.airbnb.com/rooms/918004230955889147,Rental unit in Milano · ★New · 2 bedrooms · 3 ...,Hintown,0,1
7966,916561513053933712,https://www.airbnb.com/rooms/916561513053933712,Rental unit in Milano · ★New · 1 bedroom · 1 b...,Simone,0,84
7967,914965997042368609,https://www.airbnb.com/rooms/914965997042368609,Rental unit in Milano · ★New · 1 bedroom · 1 b...,Mahnoosh,0,61
7968,917065365885178621,https://www.airbnb.com/rooms/917065365885178621,Rental unit in Milano · ★New · 2 bedrooms · 1 ...,Silvia,0,130


In [21]:
# SQL query to see who has the highest monthly revenues
pd.read_sql('SELECT id, listing_url, name, host_name, price,'
            '30-availability_30 AS days_booked,'
            '(30-availability_30)*price AS revenues FROM milan ORDER BY revenues DESC LIMIT 10',
            engine)

,id,listing_url,name,host_name,price,days_booked,revenues
0,51863754,https://www.airbnb.com/rooms/51863754,Condo in Milano · 1 bedroom · 1 bath,Una Esperienze,999,30.0,29970.0
1,549152546766700702,https://www.airbnb.com/rooms/549152546766700702,Condo in Milano · 1 bedroom · 1 bed · 1 bath,Una Esperienze,999,30.0,29970.0
2,863610123219134402,https://www.airbnb.com/rooms/863610123219134402,Rental unit in Milano · 1 bedroom · 1 bed · 1 ...,Ilia,999,30.0,29970.0
3,859294043937656110,https://www.airbnb.com/rooms/859294043937656110,Rental unit in Milano · 1 bedroom · 1 bed · 1....,Selva,992,30.0,29760.0
4,22708045,https://www.airbnb.com/rooms/22708045,Rental unit in Milano · Studio · 2 beds · 1 bath,Daniela,950,30.0,28500.0
5,29959334,https://www.airbnb.com/rooms/29959334,Rental unit in Milano · 1 bedroom · 1 bed · 1 ...,Giuseppe,950,30.0,28500.0
6,39550934,https://www.airbnb.com/rooms/39550934,Rental unit in Milano · 2 bedrooms · 3 beds · ...,Giorgio,950,30.0,28500.0
7,634455278163373759,https://www.airbnb.com/rooms/634455278163373759,Condo in Milano · ★4.92 · 3 bedrooms · 3 beds ...,Prestige,950,30.0,28500.0
8,10982650,https://www.airbnb.com/rooms/10982650,Loft in Milano · Studio · 1 bed · 2.5 baths,Max,900,30.0,27000.0
9,17809188,https://www.airbnb.com/rooms/17809188,Rental unit in Milano · 1 bedroom · 1 bed · 2 ...,Marilinda,900,30.0,27000.0


In [30]:
# SQL query to see who has the lowest monthly revenues with at least 1 day booked
pd.read_sql('SELECT id, listing_url, name, host_name, price,'
            '30-availability_30 AS days_booked,'
            '(30-availability_30)*price AS revenues FROM milan WHERE 30-availability_30 >= 1 ORDER BY revenues ASC LIMIT 10',
            engine)

,id,listing_url,name,host_name,price,days_booked,revenues
0,8696049,https://www.airbnb.com/rooms/8696049,Rental unit in Milano · 1 bedroom · 2 beds · 1...,Antonia,1,1.0,1.0
1,23765630,https://www.airbnb.com/rooms/23765630,Loft in Milano · ★5.0 · 2 bedrooms · 4 beds · ...,Sara,1,1.0,1.0
2,26374702,https://www.airbnb.com/rooms/26374702,Condo in Milano · 1 bedroom · 1 bed · 1 bath,Cinzia,1,1.0,1.0
3,52314044,https://www.airbnb.com/rooms/52314044,Rental unit in Milano · 1 bedroom · 1 bed · 1 ...,Duomo Luxury Apartments,1,1.0,1.0
4,52314048,https://www.airbnb.com/rooms/52314048,Rental unit in Milano · 1 bedroom · 1 bed · 1 ...,Duomo Luxury Apartments,1,1.0,1.0
5,636046255828034867,https://www.airbnb.com/rooms/636046255828034867,Home in Milano · 1 bedroom · 2 beds · 1 bath,Travelnest,1,1.0,1.0
6,636046258124500642,https://www.airbnb.com/rooms/636046258124500642,Rental unit in Milano · 3 bedrooms · 3 beds · ...,Travelnest,1,1.0,1.0
7,826148349614272727,https://www.airbnb.com/rooms/826148349614272727,Rental unit in Milano · ★5.0 · 1 bedroom · 2 b...,Maria Grazia,1,1.0,1.0
8,853664039537902978,https://www.airbnb.com/rooms/853664039537902978,Rental unit in Milano · 2 bedrooms · 2 beds · ...,Midori,1,1.0,1.0
9,864949219616772810,https://www.airbnb.com/rooms/864949219616772810,Rental unit in Milano · 2 bedrooms · 2 beds · ...,Francesco Antonio,1,1.0,1.0


In [37]:
# SQL query to see who has the highest review score value and the highest price for night

pd.read_sql('SELECT id, listing_url, name, host_name,review_scores_value , price FROM milan ORDER BY review_scores_value DESC , price DESC LIMIT 10', engine)

,id,listing_url,name,host_name,review_scores_value,price
0,759260243762860753,https://www.airbnb.com/rooms/759260243762860753,Rental unit in Milano · 3 bedrooms · 4 beds · ...,Aaron,5.0,995
1,32535420,https://www.airbnb.com/rooms/32535420,Rental unit in Milano · 1 bedroom · 4 beds · 1...,Sara,5.0,990
2,7640976,https://www.airbnb.com/rooms/7640976,Rental unit in Milano · ★5.0 · 2 bedrooms · 2 ...,Adele,5.0,99
3,17544479,https://www.airbnb.com/rooms/17544479,Rental unit in Milano · ★4.75 · 1 bedroom · 2 ...,Gianpiero,5.0,99
4,17846540,https://www.airbnb.com/rooms/17846540,Rental unit in Milano · 1 bedroom · 1 bed · 1 ...,Daniela Marcella,5.0,99
5,27723922,https://www.airbnb.com/rooms/27723922,Rental unit in Milano · 1 bedroom · 1 bed · 2 ...,Walter,5.0,99
6,28536904,https://www.airbnb.com/rooms/28536904,Rental unit in Milano · ★5.0 · 2 bedrooms · 3 ...,Andrea,5.0,99
7,32623946,https://www.airbnb.com/rooms/32623946,Rental unit in Milano · 1 bedroom · 3 beds · 1...,Pamela,5.0,99
8,36203995,https://www.airbnb.com/rooms/36203995,Rental unit in Milano · ★5.0 · 1 bedroom · 1 b...,Federico,5.0,99
9,39252831,https://www.airbnb.com/rooms/39252831,Rental unit in Milano · ★5.0 · 1 bedroom · 2 b...,Ombretta,5.0,99


In [39]:
# SQL query to find best rooms for a night under 50 Euros

pd.read_sql('SELECT id, listing_url, name, host_name,review_scores_value , price FROM milan WHERE price < 50 ORDER BY review_scores_value DESC , price DESC LIMIT 50', engine)

,id,listing_url,name,host_name,review_scores_value,price
0,21903718,https://www.airbnb.com/rooms/21903718,Rental unit in Milano · 2 bedrooms · 2 beds · ...,Maria,5.0,9
1,38663139,https://www.airbnb.com/rooms/38663139,Rental unit in Milano · 1 bedroom · 1 bed · 2 ...,Edoardo,5.0,5
2,746168462892951033,https://www.airbnb.com/rooms/746168462892951033,Home in Milano · ★5.0 · 1 bedroom · 1 bed · 1 ...,I-Host,5.0,5
3,5080494,https://www.airbnb.com/rooms/5080494,Rental unit in Milano · 1 bedroom · 2 beds · 2...,Alessandra,5.0,49
4,20549408,https://www.airbnb.com/rooms/20549408,Rental unit in Milano · ★5.0 · 1 bedroom · 1 b...,Fabrizio,5.0,49
5,22244736,https://www.airbnb.com/rooms/22244736,Rental unit in Milano · 1 bedroom · 1 bed · 1....,Alessio,5.0,49
6,26750044,https://www.airbnb.com/rooms/26750044,Condo in Milano · ★5.0 · 1 bedroom · 1 bed · 1...,Alessandro,5.0,49
7,30562371,https://www.airbnb.com/rooms/30562371,Rental unit in Milano · 1 bedroom · 1 bed · 1 ...,Rosanna,5.0,49
8,33514317,https://www.airbnb.com/rooms/33514317,Rental unit in Milano · ★4.91 · 1 bedroom · 1 ...,Mario,5.0,49
9,37647967,https://www.airbnb.com/rooms/37647967,Rental unit in Milano · 1 bedroom · 1 bed · 1....,Luca,5.0,49
